# SYSTE-26 Unified Workflow: Molecular Generation → Blend Optimization

This notebook demonstrates the complete end-to-end workflow:

1. **Generate novel molecules** using genetic algorithms
2. **Convert molecules to components** compatible with the blend optimizer
3. **Optimize fuel blends** using the generated components
4. **Visualize and analyze results**


In [ ]:
# Import required modules
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw

# Import molecular generator
from molecular_generator import (
    load_fragments,
    load_fragments_streaming,
    preprocess_gdb11,
    build_fragment_cache,
    evolve_with_fragments,
    molecules_to_component_csv,
)

# Import blend optimizer
from fuel_engine import load_components, greedy_then_refine

print("✅ All modules imported successfully")


### Optional: Preprocess GDB-11 Database

If you have gdb11.tgz but haven't preprocessed it yet, you can do so here:

```python
# Preprocess GDB-11 to extract fuel-relevant fragments (C/H/O only)
# This only needs to be done once - the preprocessed file can be reused

gdb11_input = project_root / "gdb11.tgz"
output_file = project_root / "data" / "fragments" / "gdb11_fuel_filtered.smi"

if gdb11_input.exists() and not output_file.exists():
    print("🔄 Preprocessing GDB-11 database...")
    count = preprocess_gdb11(
        input_file=str(gdb11_input),
        output_file=str(output_file),
        max_fragments=None,  # Process all fragments
        progress_interval=100000
    )
    print(f"✅ Preprocessing complete! Wrote {count:,} fuel-relevant fragments")
else:
    print("ℹ️ GDB-11 preprocessing skipped (file already exists or gdb11.tgz not found)")
```


## Step 1: Load Fragment Library

Load molecular fragments from the GDB-11 database or other sources.

**GDB-11 Integration**: The system now supports GDB-11 (gdb11.tgz), which contains molecules with up to 11 atoms and is better suited for fuel applications. You can:
- Use preprocessed GDB-11 files (recommended for faster loading)
- Stream directly from gdb11.tgz archives
- Use custom fragment files


In [ ]:
# Load fragments - supports multiple sources
# Option 1: Preprocessed GDB-11 file (recommended)
preprocessed_file = project_root / "data" / "fragments" / "gdb11_fuel_filtered.smi"
gdb11_tgz = project_root / "gdb11.tgz"
fragment_file = project_root / "data" / "fragments" / "high_entropy_fragments.txt"

if preprocessed_file.exists():
    # Use preprocessed GDB-11 file (fastest)
    print("📦 Loading preprocessed GDB-11 fragments...")
    fragments = load_fragments(str(preprocessed_file), max_fragments=10000)
    print(f"✅ Loaded {len(fragments):,} fragments from preprocessed file")
elif gdb11_tgz.exists():
    # Stream directly from GDB-11 archive (slower but no preprocessing needed)
    print("📦 Loading fragments from GDB-11 archive (gdb11.tgz)...")
    print("   Building cache of fuel-relevant fragments...")
    from molecular_generator.fragments import is_fuel_relevant_fragment
    fragments = build_fragment_cache(
        str(gdb11_tgz),
        cache_size=10000,
        filter_func=is_fuel_relevant_fragment
    )
    print(f"✅ Loaded {len(fragments):,} fragments from GDB-11 archive")
elif fragment_file.exists():
    # Use custom fragment file
    fragments = load_fragments(str(fragment_file), max_fragments=10000)
    print(f"✅ Loaded {len(fragments):,} fragments from custom file")
else:
    # Fallback: use a small set of example fragments
    fragments = [
        "CC", "CCO", "CCC", "C=O", "CO", "OC", "C(C)O", "CC(=O)O",
        "C1CC1", "C1CCC1", "CCCC", "CCCCC", "CCOC", "CCCO"
    ]
    print(f"⚠️ Using fallback fragments: {len(fragments)} fragments")
    print("💡 Tip: To use GDB-11, download gdb11.tgz to project root and run:")
    print("   python scripts/preprocess_gdb11.py --input gdb11.tgz --output data/fragments/gdb11_fuel_filtered.smi")


## Step 2: Generate Novel Molecules

Use genetic algorithm to evolve fuel molecules from fragments.


In [ ]:
# Evolve molecules (smaller run for demonstration)
print("🔄 Starting molecular evolution...")
evolution_df, history = evolve_with_fragments(
    fragments=fragments,
    generations=20,  # Reduced for demo (use 100-1000 for production)
    population_size=30,
    survivors=5,
    n_frags_per_molecule=3,
    verbose=True
)

print(f"\n✅ Evolution complete! Generated {len(evolution_df)} molecules")
print(f"\nTop 5 molecules by fitness:")
print(evolution_df.head())


## Step 3: Visualize Evolution Progress


In [ ]:
# Plot fitness over generations
if len(history) > 0:
    fitness_history = [h['Fitness'] for h in history]
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_history, marker='o')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.title('Fitness Evolution Over Generations')
    plt.grid(True)
    plt.show()
    
    print(f"Best fitness achieved: {max(fitness_history):.2f}")


## Step 4: Select Top Molecules

Select the best molecules for blend optimization.


In [ ]:
# Select top molecules
top_molecules = evolution_df.nlargest(10, 'Fitness')['Best_SMILES'].tolist()

print(f"Selected {len(top_molecules)} top molecules:")
for i, smiles in enumerate(top_molecules[:5], 1):
    print(f"{i}. {smiles}")

# Visualize top molecules
if top_molecules:
    mols = [Chem.MolFromSmiles(s) for s in top_molecules[:6] if Chem.MolFromSmiles(s)]
    if mols:
        img = Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(200, 200))
        display(img)


## Step 5: Convert Molecules to Components

Convert generated molecules to component format compatible with the blend optimizer.


In [ ]:
# Convert molecules to component CSV
output_csv = project_root / "data" / "components_generated.csv"

components_df = molecules_to_component_csv(
    molecules=top_molecules,
    output_path=str(output_csv),
    cost_eur_L=2.0,  # Estimated cost for novel molecules
    max_vol_frac=0.1,  # Limit novel components to 10% of blend
    is_novel=1,  # Mark as novel components
    waste_credit_eur_L=0.1  # Small waste credit for novel components
)

print(f"\n✅ Converted {len(components_df)} molecules to components")
print("\nSample component properties:")
print(components_df[['id', 'name', 'RON', 'LHV_MJ_kg', 'PMI', 'cost_eur_L']].head())


## Step 6: Load Base Components and Combine

Load existing component library and combine with generated molecules.


In [ ]:
# Load base components
base_components_file = project_root / "data" / "components.csv"

if base_components_file.exists():
    base_components = load_components(str(base_components_file))
    print(f"✅ Loaded {len(base_components)} base components")
    
    # Combine with generated components
    all_components = pd.concat([base_components, components_df], ignore_index=True)
    print(f"✅ Total components: {len(all_components)} ({len(base_components)} base + {len(components_df)} generated)")
else:
    print("⚠️ Base components file not found, using only generated components")
    all_components = components_df


## Step 7: Optimize Fuel Blends

Use the blend optimizer to find optimal fuel blends using the generated components.


In [ ]:
# Run blend optimization
print("🔄 Optimizing fuel blends...")

results = greedy_then_refine(
    all_components,
    K=5,  # Maximum components in blend
    fuel_type='gasoline',
    strict=True,
    allow_novel=True,  # Allow novel components
    max_novel=0.1,  # Maximum 10% novel components
    min_component_frac=0.01  # Minimum 1% per component
)

print(f"\n✅ Optimization complete! Generated {len(results)} blend candidates")

if results:
    best = results[0]
    print(f"\n🏆 Best blend score: {best['score']:.4f}")
    print(f"   Relative score: {best.get('relative_score', 'N/A')}")
    print(f"\n   Components:")
    for comp in best['components']:
        print(f"     - {comp['id']}: {comp['vol_frac']:.1%}")
    
    print(f"\n   Properties:")
    props = best['properties']
    print(f"     - RON: {props.get('RON', 'N/A'):.1f}")
    print(f"     - LHV (vol): {props.get('LHV_vol', 'N/A'):.2f} MJ/L")
    print(f"     - PMI: {props.get('PMI', 'N/A'):.1f}")
    print(f"     - Cost: {props.get('cost_L', 'N/A'):.2f} EUR/L")


## Step 8: Analyze Results

Compare top blend candidates and visualize their properties.


In [ ]:
# Create results DataFrame
if results:
    results_data = []
    for i, r in enumerate(results[:10], 1):  # Top 10
        results_data.append({
            'Rank': i,
            'Score': r['score'],
            'Relative_Score': r.get('relative_score', 0),
            'RON': r['properties'].get('RON', 0),
            'LHV_vol': r['properties'].get('LHV_vol', 0),
            'PMI': r['properties'].get('PMI', 0),
            'Cost_EUR_L': r['properties'].get('cost_L', 0),
            'Novel_Fraction': r.get('novel_fraction', 0),
            'Num_Components': len(r['components'])
        })
    
    results_df = pd.DataFrame(results_data)
    print("\n📊 Top 10 Blend Candidates:")
    print(results_df.to_string(index=False))
    
    # Visualize key properties
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    axes[0, 0].scatter(results_df['RON'], results_df['Score'], alpha=0.6)
    axes[0, 0].set_xlabel('RON')
    axes[0, 0].set_ylabel('Score')
    axes[0, 0].set_title('Score vs RON')
    axes[0, 0].grid(True)
    
    axes[0, 1].scatter(results_df['PMI'], results_df['Score'], alpha=0.6)
    axes[0, 1].set_xlabel('PMI')
    axes[0, 1].set_ylabel('Score')
    axes[0, 1].set_title('Score vs PMI')
    axes[0, 1].grid(True)
    
    axes[1, 0].scatter(results_df['Cost_EUR_L'], results_df['Score'], alpha=0.6)
    axes[1, 0].set_xlabel('Cost (EUR/L)')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_title('Score vs Cost')
    axes[1, 0].grid(True)
    
    axes[1, 1].bar(range(len(results_df)), results_df['Novel_Fraction'])
    axes[1, 1].set_xlabel('Rank')
    axes[1, 1].set_ylabel('Novel Fraction')
    axes[1, 1].set_title('Novel Component Usage')
    axes[1, 1].grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()


## Step 9: Home-Synthesis Mode

Generate molecules optimized for home synthesis feasibility.

This mode:
- Uses constrained fragment library (home-accessible building blocks)
- Prioritizes synthesis feasibility alongside fuel properties
- Provides synthesis route predictions
- Includes stability testing metadata


In [ ]:
# Import home synthesis modules
from molecular_generator.home_synthesis_fragments import HOME_SYNTHESIS_FRAGMENTS
from molecular_generator.generator import evolve_home_synthesizable_molecules
from molecular_generator.synthesis import suggest_synthesis_route, format_synthesis_instructions

# Generate home-synthesizable molecules
print("🔄 Generating home-synthesizable molecules...")
home_synth_df, home_synth_history = evolve_home_synthesizable_molecules(
    fragments=HOME_SYNTHESIS_FRAGMENTS,
    generations=30,  # Reduced for demo
    population_size=30,
    survivors=5,
    n_frags_per_molecule=2,  # Simpler molecules
    min_feasibility=60.0,  # Minimum 60% feasibility
    synthesis_weight=0.5,  # 50% weight on synthesis
    fuel_weight=0.3,  # 30% weight on fuel properties
    verbose=True
)

print(f"\n✅ Generated {len(home_synth_df)} home-synthesizable molecules")
if len(home_synth_df) > 0:
    print(f"\nTop 5 molecules by synthesis feasibility:")
    top_synth = home_synth_df.nlargest(5, 'Synthesis_Feasibility')
    for idx, row in top_synth.iterrows():
        print(f"  {row['Best_SMILES']}: Feasibility={row['Synthesis_Feasibility']:.1f}, "
              f"Complexity={row['Synthesis_Complexity']:.1f}")


## Step 10: View Synthesis Routes

Display synthesis instructions for top molecules.


In [ ]:
# Display synthesis routes for top molecules
if len(home_synth_df) > 0:
    top_molecules_synth = home_synth_df.nlargest(5, 'Synthesis_Feasibility')
    
    for idx, row in top_molecules_synth.iterrows():
        smiles = row['Best_SMILES']
        print(f"\n{'='*60}")
        print(f"Molecule: {smiles}")
        print(f"Feasibility: {row['Synthesis_Feasibility']:.1f}/100")
        print(f"Complexity: {row['Synthesis_Complexity']:.1f}/10")
        print(f"Fitness: {row['Fitness']:.2f}")
        print(f"\nSynthesis Route:")
        
        route = suggest_synthesis_route(smiles)
        instructions = format_synthesis_instructions(route)
        print(instructions)
        print()


## Step 11: Convert to Components with Synthesis Metadata

Convert home-synthesizable molecules to components with full synthesis and stability metadata.


In [ ]:
# Convert home-synthesizable molecules to components
if len(home_synth_df) > 0:
    top_home_molecules = home_synth_df.nlargest(10, 'Synthesis_Feasibility')['Best_SMILES'].tolist()
    
    home_components_df = molecules_to_component_csv(
        molecules=top_home_molecules,
        output_path=str(project_root / "data" / "components_home_synthesis.csv"),
        cost_eur_L=1.5,  # Lower cost estimate for home synthesis
        max_vol_frac=0.15,  # Allow up to 15% in blend
        is_novel=1,
        waste_credit_eur_L=0.1
    )
    
    print(f"\n✅ Converted {len(home_components_df)} molecules to components")
    print("\nSample component with synthesis metadata:")
    sample_cols = ['id', 'name', 'synthesis_feasibility', 'synthesis_difficulty', 
                   'oxidation_susceptibility', 'RON', 'LHV_MJ_kg']
    available_cols = [c for c in sample_cols if c in home_components_df.columns]
    print(home_components_df[available_cols].head(3))


## Step 12: Stability Testing Candidates

Select molecules for stability testing based on oxidation susceptibility.


In [ ]:
# Select stability testing candidates
if len(home_components_df) > 0:
    # High oxidation risk (for testing)
    high_oxidation = home_components_df.nlargest(5, 'oxidation_susceptibility')
    
    # Low oxidation risk (for controls)
    low_oxidation = home_components_df.nsmallest(5, 'oxidation_susceptibility')
    
    print("🔬 Stability Testing Candidates")
    print("\nHigh Oxidation Risk (Test Group):")
    print(high_oxidation[['name', 'oxidation_susceptibility', 'synthesis_feasibility', 
                          'synthesis_difficulty']].to_string(index=False))
    
    print("\nLow Oxidation Risk (Control Group):")
    print(low_oxidation[['name', 'oxidation_susceptibility', 'synthesis_feasibility', 
                          'synthesis_difficulty']].to_string(index=False))
    
    # Save stability testing candidates
    stability_candidates = pd.concat([
        high_oxidation.assign(test_group='high_oxidation'),
        low_oxidation.assign(test_group='low_oxidation')
    ])
    stability_candidates.to_csv(
        project_root / "data" / "stability_testing_candidates.csv",
        index=False
    )
    print(f"\n✅ Saved {len(stability_candidates)} stability testing candidates")


## Summary

This workflow demonstrates:

1. ✅ **Molecular Generation**: Evolved novel fuel molecules using genetic algorithms
2. ✅ **Component Conversion**: Converted molecules to blend optimizer format
3. ✅ **Blend Optimization**: Found optimal fuel blends using generated components
4. ✅ **Analysis**: Visualized and compared results

### Next Steps

- Run longer evolution (100-1000 generations) for better molecules
- Experiment with different fitness functions
- Try different fuel types (diesel, jet fuel)
- Use the optimized blends in the command-line engine: `python engine.py --components data/components_generated.csv --novel data/components_generated.csv --allow-novel 1 --fuel-type gasoline`
